# Fibonacci

1, 1, 2, 3, 5, 8, 13, 21...

In [1]:
# timer
def timer_factory(times=10**3):
    def interface(func):
        from functools import wraps
        from time import perf_counter

        @wraps(func)
        def timer(*args, **kwargs):
            s = perf_counter()
            for i in range(times):
                result = func(*args, **kwargs)
            e = perf_counter()
            elapsed = e - s
            print('Executed {} times, average took: {:05.3f}us'.format(times, 10**6 * elapsed / times))

            return result
        return timer
    return interface

### recursion

In [2]:
def fib(n):
    if n <= 2:
        return 1
    return fib(n-1) + fib(n-2)

@timer_factory(1000)
def fib_run(n):
    fib(n)

In [3]:
fib_run(20)

Executed 1000 times, average took: 1493.964us


### loop

In [4]:
@timer_factory(1000)
def fib_loop(n):
    x = 1
    y = 1
    for i in range(3, n+1):
        x, y = y, x + y
    return y

In [5]:
fib_loop(20)

Executed 1000 times, average took: 1.583us


6765

### reduce

<pre>
previous value: (a, b)
new value: (a+b, a)
</pre>

In [6]:
from functools import reduce
@timer_factory(1000)
def fib_reduce(n):
    initial = (1, 0)
    dummy = range(n - 1)  # make reduce work n times
    result = reduce(lambda prev, n: (prev[0] + prev[1], prev[0]), 
                    dummy,
                    initial)
    return result[0]

In [7]:
fib_reduce(20)

Executed 1000 times, average took: 14.378us


6765

## improve resursion using cache

### native

In [8]:
def fib(n):
    cache = {1: 1, 2: 1}
    def inner(n):       
        if n not in cache:
            cache[n] = inner(n-1) + inner(n-2)
        return cache[n]
    return inner

@timer_factory(1000)
def fib_1_test(n):
    fib(n)

In [9]:
fib_1_test(20)

Executed 1000 times, average took: 1.789us


### using lru_cache

In [10]:
from functools import lru_cache

@lru_cache(maxsize=8)
def fib(n):
    if n <= 2:
        return 1
    return fib(n-1) + fib(n-2)

@timer_factory(1000)
def fib_2_test(n):
    fib(n)

In [11]:
fib_2_test(20)

Executed 1000 times, average took: 0.224us


### using custom cache

In [12]:
def use_cache(fib):
    cache = {1: 1, 2: 1}
    def interface(n):       
        if n not in cache:
            cache[n] = fib(n)
        return cache[n]
    return interface

@use_cache
def fib(n):
    if n <= 2:
        return 1
    return fib(n-1) + fib(n-2)

@timer_factory(1000)
def fib_3_test(n):
    fib(n)

In [13]:
fib_3_test(20)

Executed 1000 times, average took: 0.417us


## 1000 time test conclusion
1. recursion with cache
2. loop
3. reduce
4. recursion without cache

1 > 2 > 3 >>>> 4